<a href="https://colab.research.google.com/github/dhrits/LLM-Engineering-Foundations-to-SLMs/blob/main/05_Next-Token-Prediction/Decoding_From_Logits_to_the_Speculative_Decoding_and_Guard_Rails_Hardmode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decoding: From Logits to the Speculative Decoding and Guard Rails

We've covered everything from loss to logits - and now we're going to dive in to how we select the next token in details, and all the options we can use to do it!

Breakout Room #1: Logits to Tokens
- Task 1: Dependencies
- Task 2: Generating Tokens!
  - 🏗️ Activity #1:
- Task 3: Data Preprocessing
  - ❓Question #1
- Task 4: Alternate Decoding Examples:
  - 👪❓ Discussion Question #1

Breakout Room #2: Speculative Decoding and Guard Rails
- Task 5: Speculative Decoding
  - ❓ Discussion Question #2
- Task 6: Guard Rails
  - 👪❓ Discussion Question #2

## Task 1: Dependencies

Today we'll be using a classic minamalist implementation of a decoder-only transformer model called `nanoGPT`, built by the one-and-only Andrej Karpathy - found [here](https://github.com/karpathy/nanoGPT/tree/master)!

It does require a few dependencies - though most are covered by the default Colab environment.

> NOTE: You will need to make sure you're in a GPU enabled environment for effective use of this notebook.

In [1]:
!pip install -qU datasets tiktoken wandb tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!git clone https://github.com/karpathy/nanoGPT.git

Cloning into 'nanoGPT'...
remote: Enumerating objects: 682, done.
remote: Total 682 (delta 0), reused 0 (delta 0), pack-reused 682 (from 1)
Receiving objects: 100% (682/682), 952.47 KiB | 41.41 MiB/s, done.
Resolving deltas: 100% (385/385), done.


In [3]:
%cd nanoGPT

/content/nanoGPT


## Task 2: Generating Tokens!

Let's just try to do some inference and see what happens before we dig in.

In [ ]:
!python sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
2024-12-07 21:32:11.131461: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:32:11.148441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:32:11.169369: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:32:11.175681: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cu

You'll notice that we pass *in* text - and we receive *back* text from our model.

##### 🏗️ Activity #1:

*Where* (in the architecture) exactly does NanoGPT go "from logits to predicted token"? Provide a screenshot from [this visualization](https://bbycroft.net/llm)!

Based on the visualization, this is happening in "Logits Softmax" layer where the logits are converted to a probability distribution using softmax and then sampled from using different strategies. Finally, the sampled index is converted back to text using the tokenizer.

![visualization](https://github.com/dhrits/LLM-Engineering-Foundations-to-SLMs/blob/main/05_Next-Token-Prediction/LogitsSoftmax.png?raw=1)

## Task 3: How Does the LLM Generate Tokens

So, we pas in text - and get text back - but how do we actually generate each token?

You might have heard the term "auto-regressive" or "causal" kicking around when reading about LLMs - and what those terms, in a simplified sense, mean is straightfoward enough:

- They take an input, and generate a single token
- They append that token to the input and repeat this process for as long as we want it to repeat (or use heuristics to determine when to stop, such as when we see a stop token)

Let's take a look at the function that does this in the `nanoGPT` repository.



```python
@torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx
```

### What is a Logit?!

Technically - a logit is a "raw unnormalized score".

However, we can think of them as scores for each token in our vocabulary. These scores aren't probabilities in and of themselves - but they can be easily converted to probabilities through the softmax function.

### What is Temperature Doing?!

While something like `top_k` makes intuitive sense - what in the heck is temperature doing here?

In order to understand - let's look at a few examples!

Starting with an easy `temperature = 1.0`.

> NOTE: We'll also define our softmax function!

In [ ]:
def softmax(x):
    return(np.exp(x - np.max(x)) / np.exp(x - np.max(x)).sum())

In [ ]:
import numpy as np

temperature = 1.0

logits = np.array([6, 2, 7, 0.1, -8, 9])

temp_scaled_logits = logits / temperature
print(f"Scaled Logits: {temp_scaled_logits}")

softmaxed_logits = softmax(temp_scaled_logits)
print(f"Softmax-ed Logits: {softmaxed_logits}")

Scaled Logits: [ 6.   2.   7.   0.1 -8.   9. ]
Softmax-ed Logits: [4.19729385e-02 7.68761185e-04 1.14094276e-01 1.14982549e-04
 3.49017038e-08 8.43049007e-01]


As you can see - our logits are not changed, and our softmax output has quite a bit of variety - from `e-08` to `e-1`, meaning that our index with the score `9` is most likely to be selected, but it's not absurdly likely.

Let's look at an example with a very low temperature next!

In [ ]:
temperature = 0.1

logits = np.array([6, 2, 7, 0.1, -8, 9])

temp_scaled_logits = logits / temperature
print(f"Scaled Logits: {temp_scaled_logits}")

softmaxed_logits = softmax(temp_scaled_logits)
print(f"Softmax-ed Logits: {softmaxed_logits}")

Scaled Logits: [ 60.  20.  70.   1. -80.  90.]
Softmax-ed Logits: [9.35762295e-14 3.97544973e-31 2.06115362e-09 2.22736356e-39
 1.47889750e-74 9.99999998e-01]


As you can see - now that we changed our temperature to be very low - the index with score `9` is *vastly* more likely than any other option.

This is the idea that a low (<1) temperature value will scale our logits to be larger - resulting in a sharper probability distribution after softmax.

Let's look at a final example with a higher temperature.

In [ ]:
temperature = 100

logits = np.array([6, 2, 7, 0.1, -8, 9])

temp_scaled_logits = logits / temperature
print(f"Scaled Logits: {temp_scaled_logits}")

softmaxed_logits = softmax(temp_scaled_logits)
print(f"Softmax-ed Logits: {softmaxed_logits}")

Scaled Logits: [ 0.06   0.02   0.07   0.001 -0.08   0.09 ]
Softmax-ed Logits: [0.17201758 0.16527268 0.17374639 0.16216214 0.1495449  0.1772563 ]


In [ ]:
!python sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100 --temperature=100.0

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
Overriding: temperature = 100.0
2024-12-07 21:33:17.783616: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:33:17.800545: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:33:17.821367: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:33:17.827780: E external/local_xla/xla/stream_executor/cuda/cuda_bla

Now we can see that, while our index with score `9` is still the most likely - we can see that the probabilities are much closer together!

##### ❓ Question #1:

Why is the softmax operation so important for decoding?

The softmax operation converts the raw unnormalized model outputs or logits int a probability distribution which can be sampled from. Once we have this distribution, we can apply various sampling strategies from statistics to output the next token.

## Task 4: Alternate Decoding Examples

Let's look at a few other methods we could use to go from our logits to some text!

We'll look at each component of the following script for each different method in detail as we go through them!

### Full Modified Generation Script

In [ ]:
new_sample_script = r'''
"""
Sample from a trained model with various decoding strategies, including repetition penalty
"""
import os
import pickle
from contextlib import nullcontext
import torch
import torch.nn.functional as F
import tiktoken
from model import GPTConfig, GPT

# -----------------------------------------------------------------------------
init_from = 'resume' # either 'resume' (from an out_dir) or a gpt2 variant (e.g. 'gpt2-xl')
out_dir = 'out' # ignored if init_from is not 'resume'
start = "\n" # or "<|endoftext|>" or etc. Can also specify a file, use as: "FILE:prompt.txt"
num_samples = 10 # number of samples to draw
max_new_tokens = 500 # number of tokens generated in each sample
temperature = 0.8 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 200 # retain only the top_k most likely tokens, clamp others to have 0 probability
top_p = 0.0 # If set, use nucleus sampling
beam_width = 0 # If set, use beam search with this width
greedy = False # use greedy decoding
repetition_penalty = 1.0 # 1.0 means no penalty, > 1.0 discourages repetition, < 1.0 encourages repetition
seed = 1337
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1', etc.
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
compile = False # use PyTorch 2.0 to compile the model to be faster
exec(open('configurator.py').read()) # overrides from command line or config file
# -----------------------------------------------------------------------------

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# model
if init_from == 'resume':
    # init from a model saved in a specific directory
    ckpt_path = os.path.join(out_dir, 'ckpt.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    gptconf = GPTConfig(**checkpoint['model_args'])
    model = GPT(gptconf)
    state_dict = checkpoint['model']
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
elif init_from.startswith('gpt2'):
    # init from a given GPT-2 model
    model = GPT.from_pretrained(init_from, dict(dropout=0.0))

model.eval()
model.to(device)
if compile:
    model = torch.compile(model) # requires PyTorch 2.0 (optional)

# look for the meta pickle in case it is available in the dataset folder
load_meta = False
if init_from == 'resume' and 'config' in checkpoint and 'dataset' in checkpoint['config']: # older checkpoints might not have these...
    meta_path = os.path.join('data', checkpoint['config']['dataset'], 'meta.pkl')
    load_meta = os.path.exists(meta_path)
if load_meta:
    print(f"Loading meta from {meta_path}...")
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    # TODO want to make this more general to arbitrary encoder/decoder schemes
    stoi, itos = meta['stoi'], meta['itos']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])
else:
    # ok let's assume gpt-2 encodings by default
    print("No meta.pkl found, assuming GPT-2 encodings...")
    enc = tiktoken.get_encoding("gpt2")
    encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
    decode = lambda l: enc.decode(l)

# Top-P (nucleus) sampling
def top_p_sampling(logits, p):
    probs = F.softmax(logits, dim=-1)
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0
    indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
    probs = probs.masked_fill(indices_to_remove, 0.0)
    return torch.multinomial(probs, num_samples=1)

# Beam Search with Repetition Penalty
def beam_search(model, x, max_new_tokens, beam_width, repetition_penalty):
    beams = [(x, 0)]
    for _ in range(max_new_tokens):
        candidates = []
        for beam_idx, (sequence, log_prob) in enumerate(beams):
            if sequence.size(1) > model.config.block_size:
                idx_cond = sequence[:, -model.config.block_size:]
            else:
                idx_cond = sequence
            logits, _ = model(idx_cond)
            logits = logits[:, -1, :]

            # Apply repetition penalty
            if repetition_penalty != 1.0:
              # Get unique tokens in the generated sequence
              unique_tokens = torch.unique(x[0])

              # Create a mask for the tokens that have been used
              mask = torch.zeros_like(logits[0]).bool()
              mask[unique_tokens] = True

              # Apply the penalty only to the used tokens
              logits[0, mask] /= repetition_penalty

            probs = F.log_softmax(logits, dim=-1)
            top_probs, top_indices = probs.topk(beam_width)
            for prob, idx in zip(top_probs[0], top_indices[0]):
                new_sequence = torch.cat((sequence, idx.unsqueeze(0).unsqueeze(0)), dim=1)
                new_log_prob = log_prob + prob.item()
                candidates.append((new_sequence, new_log_prob))
        beams = sorted(candidates, key=lambda x: x[1], reverse=True)[:beam_width]
    return beams[0][0]

@torch.no_grad()
def generate(model, x, max_new_tokens, temperature=1.0, top_k=None, top_p=None, beam_width=None, repetition_penalty=1.0, greedy=False):
    print(f"Starting generation with max_new_tokens={max_new_tokens}")

    if beam_width != 0:
        print(f"Using beam search with width {beam_width}")
        return beam_search(model, x, max_new_tokens, beam_width, repetition_penalty)

    for i in range(max_new_tokens):
        if i % 100 == 0:
            print(f"Generated {i} tokens")

        if x.size(1) > model.config.block_size:
            idx_cond = x[:, -model.config.block_size:]
        else:
            idx_cond = x
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature

        # Apply repetition penalty
        if repetition_penalty != 1.0:
          # Get unique tokens in the generated sequence
          unique_tokens = torch.unique(x[0])

          # Create a mask for the tokens that have been used
          mask = torch.zeros_like(logits[0]).bool()
          mask[unique_tokens] = True

          # Apply the penalty only to the used tokens
          logits[0, mask] /= repetition_penalty

        if greedy:
            # Greedy decoding: select the token with the highest probability
            idx_next = torch.argmax(logits, dim=-1).unsqueeze(0)
        else:
            # Existing sampling methods
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            if top_p != 0.0:
                idx_next = top_p_sampling(logits, top_p)
            else:
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)

        x = torch.cat((x, idx_next), dim=1)

    print("Generation complete")
    return x

# encode the beginning of the prompt
if start.startswith('FILE:'):
    with open(start[5:], 'r', encoding='utf-8') as f:
        start = f.read()
start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            print(f"Generating sample {k+1}/{num_samples}")
            y = generate(model, x, max_new_tokens, temperature=temperature, top_k=top_k, top_p=top_p,
                         beam_width=beam_width, repetition_penalty=repetition_penalty, greedy=greedy)
            print(decode(y[0].tolist()))
            print('---------------')
'''

with open('extended_sample.py', 'w') as f:
  f.write(new_sample_script)

### Vanilla Sample Generation

```python
@torch.no_grad()
def generate(model, x, max_new_tokens, temperature=1.0, top_k=None, top_p=None, beam_width=None, repetition_penalty=1.0, greedy=False):
    print(f"Starting generation with max_new_tokens={max_new_tokens}")
    
    if beam_width != 0:
        print(f"Using beam search with width {beam_width}")
        return beam_search(model, x, max_new_tokens, beam_width, repetition_penalty)

    for i in range(max_new_tokens):
        if i % 100 == 0:
            print(f"Generated {i} tokens")
        
        if x.size(1) > model.config.block_size:
            idx_cond = x[:, -model.config.block_size:]
        else:
            idx_cond = x
        logits, _ = model(idx_cond)
        logits = logits[:, -1, :] / temperature

        # Apply repetition penalty
        if repetition_penalty != 1.0:
          # Get unique tokens in the generated sequence
          unique_tokens = torch.unique(x[0])
          
          # Create a mask for the tokens that have been used
          mask = torch.zeros_like(logits[0]).bool()
          mask[unique_tokens] = True
          
          # Apply the penalty only to the used tokens
          logits[0, mask] /= repetition_penalty

        if greedy:
            # Greedy decoding: select the token with the highest probability
            idx_next = torch.argmax(logits, dim=-1).unsqueeze(0)
        else:
            # Existing sampling methods
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            if top_p != 0.0:
                idx_next = top_p_sampling(logits, top_p)
            else:
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)

        x = torch.cat((x, idx_next), dim=1)

    print("Generation complete")
    return x
```

As we can see - if we *don't* use a `top_p`, `top_k`, or `beam_width` - we simply sample the distribution as-in.

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
2024-12-07 21:35:17.065539: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:35:17.082470: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:35:17.103361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:35:17.109737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cu

### Greedy Decoding

Greedy decoding is the simplest method for generating text from a language model. It works as follows:

1. **Prediction**: At each step, the model predicts the probability distribution for the next token.

2. **Selection**: The token with the highest probability is selected.

3. **Iteration**: This process is repeated until the desired length is reached or a stop condition is met.

Key characteristics of greedy decoding:

- **Deterministic**: Given the same input and model state, it always produces the same output.
- **Fast**: It's computationally efficient as it doesn't require sampling or complex calculations.
- **Lack of diversity**: It tends to generate repetitive and sometimes boring text, especially for longer sequences.

Let's check out the code!

```python
if greedy:
    # Greedy decoding: select the token with the highest probability
    idx_next = torch.argmax(logits, dim=-1).unsqueeze(0)
```

And see it in practice!

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100 --greedy=True

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
Overriding: greedy = True
2024-12-07 21:36:03.937973: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:36:03.954784: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:36:03.975594: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:36:03.981895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1

That is a bit repetitive, is there way we can reduce this?

### Repetition Penalty

Repetition penalty is a technique used in text generation to reduce the likelihood of a model repeating the same words or phrases too frequently. It works by dynamically adjusting the probability of tokens that have already been generated.

Key aspects of repetition penalty:

1. **Penalization**: It decreases the probability of tokens that have appeared in the generated text.
2. **Adjustable strength**: The penalty can be tuned to control the trade-off between coherence and diversity.
3. **Applicability**: It can be applied alongside other decoding methods like greedy search, beam search, or sampling techniques.

The basic idea is to divide the logits (unnormalized prediction scores) of previously generated tokens by a penalty factor.

```python
if repetition_penalty != 1.0:
    # Consider only the last 1000 tokens (adjust as needed)
    window_size = 1000
    recent_tokens = x[0, -window_size:]
    
    # Get unique tokens in the recent sequence
    unique_tokens = torch.unique(recent_tokens)
    
    # Create a mask for the tokens that have been used recently
    mask = torch.zeros_like(logits[0]).bool()
    mask[unique_tokens] = True
    
    # Apply the penalty only to the recently used tokens
    logits[0, mask] /= repetition_penalty
```

Let's see it in practice!

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100 --greedy=True --repetition_penalty=1.8

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
Overriding: greedy = True
Overriding: repetition_penalty = 1.8
2024-12-07 21:36:50.714875: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:36:50.731048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:36:50.751465: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:36:50.757667: E external/local_xla/xl

### Top-P (Nucleus) Sampling

Top-P sampling, also known as nucleus sampling, is a text generation technique that provides a balance between diversity and quality in the generated text. It works as follows:

1. **Sorting probabilities**: After the model produces the probability distribution for the next token, the probabilities are sorted in descending order.

2. **Cumulative sum**: A cumulative sum of these sorted probabilities is calculated.

3. **Probability mass selection**: A threshold p (typically between 0.9 and 1) is chosen. The smallest set of tokens whose cumulative probability exceeds p is selected.

4. **Sampling**: The next token is randomly sampled from this reduced set of tokens.

Key advantages of Top-P sampling:

- It adapts to the confidence of the model's predictions.
- It can produce more diverse outputs than methods like Top-K sampling, especially for less confident predictions.
- It helps avoid low-probability tokens while maintaining a dynamic vocabulary size.

```python
def top_p_sampling(logits, p):
    probs = F.softmax(logits, dim=-1)
    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
    sorted_indices_to_remove = cumulative_probs > p
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0
    indices_to_remove = sorted_indices_to_remove.scatter(1, sorted_indices, sorted_indices_to_remove)
    probs = probs.masked_fill(indices_to_remove, 0.0)
    return torch.multinomial(probs, num_samples=1)
```

Let's see it in action!

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100 --top_p=0.95

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
Overriding: top_p = 0.95
2024-12-07 21:37:38.146622: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:37:38.163507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:37:38.184359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:37:38.190663: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:14

Let's see this with a low temperature, and a high temperature and observe the difference!

Cold as ice:

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100 --top_p=0.95 --temperature=0.1

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
Overriding: top_p = 0.95
Overriding: temperature = 0.1
2024-12-07 21:38:25.299689: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:38:25.316511: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:38:25.337622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:38:25.343956: E external/local_xla/xla/stream

Burning hot:

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=100 --top_p=0.05 --temperature=100.0

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 100
Overriding: top_p = 0.05
Overriding: temperature = 100.0
2024-12-07 21:39:12.285171: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:39:12.302574: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:39:12.324065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:39:12.330548: E external/local_xla/xla/stre

### Beam Search

Beam search is a heuristic search algorithm commonly used in natural language processing for text generation tasks. It's an extension of greedy decoding that explores multiple possible sequences simultaneously.

Key concepts:

1. **Beam width**: The number of sequences to keep track of at each step.
2. **Expanding**: At each step, generate all possible next tokens for each sequence in the beam.
3. **Pruning**: Keep only the top-k sequences (where k is the beam width) based on their cumulative probability scores.

Algorithm overview:

1. Start with an initial sequence (usually a start token).
2. Generate the next token probabilities for each sequence in the beam.
3. Create new candidate sequences by appending each possible next token.
4. Score each candidate sequence based on its cumulative log probability.
5. Select the top-k sequences to form the new beam.
6. Repeat steps 2-5 until the desired length is reached or a stop condition is met.

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="What is the answer to life, the universe, and everything?" \
    --num_samples=1 --max_new_tokens=50 --beam_width=16 --repetition_penalty=1.8 --temperature=1.4

Overriding: init_from = gpt2-xl
Overriding: start = What is the answer to life, the universe, and everything?
Overriding: num_samples = 1
Overriding: max_new_tokens = 50
Overriding: beam_width = 16
Overriding: repetition_penalty = 1.8
Overriding: temperature = 1.4
2024-12-07 21:40:37.153087: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:40:37.170000: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:40:37.190940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:40:37.

In [ ]:
!python extended_sample.py \
    --init_from=gpt2-xl \
    --start="Why is the sky blue?" \
    --num_samples=1 --max_new_tokens=50 --beam_width=16 --repetition_penalty=1.8 --temperature=1.4

Overriding: init_from = gpt2-xl
Overriding: start = Why is the sky blue?
Overriding: num_samples = 1
Overriding: max_new_tokens = 50
Overriding: beam_width = 16
Overriding: repetition_penalty = 1.8
Overriding: temperature = 1.4
2024-12-07 21:41:41.443698: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-07 21:41:41.460560: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 21:41:41.481288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 21:41:41.487563: E external/local_xla/xla/stre

##### 👪❓ Discussion Question #2:

Which decoding method is *best*? If there is no obvious best, please list the pros/cons/use-cases for each method.

There is no obvious "best" method when it comes to sampling and it depends on the nature of the application we're building and our compute budget. Examining some pros and cons of each approach:

1. Greedy - This sampling method should typically only be used for prediction of very short sequences of tokens (ideally 1-3) because of how much it optimizes for the next token. It optimizes for the short horizon and can lead to issues with longer sequences. However, it is computationally very efficient.
2. Sampling from the distribution (no changes) - This method can be a good balance between greedy and more computationally complex methods which may be computationally expensive. If we believe that the learned distribution of the model is decent, then this method might be effective. However, it may lead to some meaningless or repetitive outputs since it has no repetition penalty.
3. top-k and top-p - Both of these methods can be a good balance between greedy and sampling. Their use can lead to fewer surprises as they still sample most likely outcomes. However, their use with a low temperature might lead to repetituion. This can be countered with repetition penalty.
4. Beam Search - This method is most interesting for predicting long sequences because it optimizes for the joint probabilty of the sequence as opposed to the next token. It may produce some surprising results but they can be countered if it is used together with other parameters like repetion penalty, top-p and topk. The downside is that it is extremely computationally expensive and can still produce some repetitive results.
5. Temperature and Repetition Penalty - These are less sampling methods but rather parameters which can be used with other methods. Use of high(er) temperature (>1) can encourage the model to be more creative. Repetition penalty can avoid repeated outputs and counter the "greedy" effect in sampling.

If I had to pick one method only (although I don't know why I would restrict myself in this way), I would probably use top-p with a temperature > 1 and some repetition penalty as a default. However, some tuning would be required depending on the model to find the best defaults.

# Breakout Room #2:

## Task 5: Speculative Decoding

Speculative decoding is a technique that aims to speed up text generation by using a smaller, faster "draft" model to predict multiple tokens at once, which are then verified by a larger, more accurate model. Here's how it works:

1. Draft Stage:

- A smaller, faster model proposes a sequence of tokens
- The draft model can generate these quickly but with lower quality
- The number of tokens generated is a tunable parameter (num_draft_tokens)


2. Verification Stage:

- A larger, more accurate model evaluates the proposed tokens
- For each token, it calculates the probability ratio between the draft and verifier models
- Tokens are accepted if the ratio meets certain criteria


3. Accept/Reject Process:

- Accepted tokens are added to the sequence
- If a token is rejected, we fall back to the verifier's prediction
- The process then continues from the last accepted token


**Key advantages**:

- Can provide significant speedup (2-3x) over traditional decoding
- Maintains quality of the larger model
- Particularly effective for long sequences

##### ⚠️ Hardmode Challenge:

Please implement speculative decoding - you can assume that `gpt-2` and `gpt2-medium` are compatible enough for this process to loosely work.

In [22]:
import torch
import torch.nn.functional as F
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import gc

class DirectSpeculativeDecoder:
    def __init__(self, draft_model_name='gpt2', verifier_model_name='gpt2-medium', num_draft_tokens=4,
                 loss_model_name='gpt2-medium'):
        """Initialize decoder with draft and verifier models"""
        # Clear CUDA cache and garbage collect
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Using device: {self.device}")

        # Set model precision
        self.dtype = torch.float16 if self.device == 'cuda' else torch.float32

        # Load models and tokenizer with proper memory handling
        print(f"Loading draft model: {draft_model_name}")
        self.draft_model = GPT2LMHeadModel.from_pretrained(
            draft_model_name,
            torch_dtype=self.dtype,
            low_cpu_mem_usage=True
        )
        self.draft_model.eval().to(self.device)

        print(f"Loading verifier model: {verifier_model_name}")
        self.verifier_model = GPT2LMHeadModel.from_pretrained(
            verifier_model_name,
            torch_dtype=self.dtype,
            low_cpu_mem_usage=True
        )
        self.verifier_model.eval().to(self.device)

        print(f"Loading loss model: {loss_model_name}")
        self.loss_model = GPT2LMHeadModel.from_pretrained(
            loss_model_name,
            torch_dtype=self.dtype,
            low_cpu_mem_usage=True
        )
        self.loss_model.eval().to(self.device)

        self.tokenizer = GPT2Tokenizer.from_pretrained(draft_model_name)
        self.num_draft_tokens = num_draft_tokens

        # Add padding token if it doesn't exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

    def _safe_model_output(self, model, input_ids):
        """Safely get model outputs with proper error handling"""
        try:
            with torch.cuda.amp.autocast():
                outputs = model(input_ids)
            return outputs
        except RuntimeError as e:
            print(f"Error during model inference: {str(e)}")
            torch.cuda.empty_cache()
            gc.collect()
            raise

    @torch.no_grad()
    def generate(self, prompt, max_new_tokens, temperature=1.0,
                 use_verifier=True, return_loss=False):
        """Generate text using speculative decoding"""
        try:
            input_ids = self.tokenizer.encode(prompt, return_tensors='pt').to(self.device)
            block_size = min(self.draft_model.config.n_ctx, self.verifier_model.config.n_ctx)
            generated_tokens = 0
            while generated_tokens < max_new_tokens:
              draft_input_ids = input_ids.clone()
              if draft_input_ids.size(1) >= block_size:
                draft_input_ids = draft_input_ids[:, -block_size:]
              initial_length = draft_input_ids.size(1)
              draft_tokens = []
              for _ in range(self.num_draft_tokens):
                logits = self._safe_model_output(self.draft_model, draft_input_ids).logits[:, -1, :] / temperature
                probs = F.softmax(logits, dim=-1)
                token = torch.multinomial(probs, num_samples=1)
                draft_input_ids = torch.cat((draft_input_ids, token), dim=1)
                draft_tokens.append(token.item())

              if not use_verifier:
                # For testing loss without using verifier
                input_ids = torch.cat([input_ids, torch.tensor([draft_tokens], dtype=input_ids.dtype, device=input_ids.device)], dim=1)
                generated_tokens += len(draft_tokens)
                continue

              # Note that the version of this algorithm in the regular notebook
              # doesn't follow the psuedocode listed above. It continues to sample
              # from the verification_probs even after a rejection. When generating
              # longer samples, this is problematic. My version below, starts
              # sampling again from the last accepted token in the event of a
              # rejection. This leads to better outputs for longer sequences.

              # Also note that the regular notebook makes use of config.max_length
              # which in the case of gpt2 config isn't really the context length.

              # Verification
              verification_temperature = temperature/1.25 # Helps for verification model
              verification_logits = self._safe_model_output(self.verifier_model, draft_input_ids).logits / verification_temperature
              verification_probs = F.softmax(verification_logits, dim=-1)

              # Accept/Reject

              for i in range(self.num_draft_tokens):
                token = draft_input_ids[0, initial_length + i].item()
                if torch.rand(1).item() < verification_probs[0, initial_length + i - 1, token].item():
                  # Accept the token
                  input_ids = torch.cat([input_ids, torch.tensor([[token]], dtype=torch.long, device=self.device)], dim=1)
                  generated_tokens += 1
                else:
                  new_token = torch.multinomial(verification_probs[:, initial_length + i - 1], 1)
                  # Use newly generated token from verification model
                  input_ids = torch.cat([input_ids, new_token], dim=1)
                  generated_tokens += 1
                  break

            output_text = self.tokenizer.decode(input_ids[0], skip_special_tokens=True)
            if return_loss:
              logits = self._safe_model_output(self.loss_model, input_ids).logits / temperature
              logits = logits[:, :-1, :].contiguous()
              input_ids = input_ids[:, 1:].contiguous()
              input_ids = input_ids.view(-1)
              logits = logits.view(-1, logits.size(-1))
              loss = F.cross_entropy(logits, input_ids)
              return output_text, loss.item()
            else:
              return output_text

        except Exception as e:
            print(f"Error during generation: {str(e)}")
            if self.device == 'cuda':
                torch.cuda.empty_cache()
            gc.collect()
            raise


Now we just need some code to actually run that beast!

In [30]:
def run_speculative_decoder(prompt, max_tokens=50, use_verifier=True, return_loss=True):
    try:
        decoder = DirectSpeculativeDecoder(
            draft_model_name='gpt2',
            verifier_model_name='gpt2-medium',
            num_draft_tokens=4
        )

        print("\nGenerating with prompt:", prompt)
        result = decoder.generate(
            prompt=prompt,
            max_new_tokens=max_tokens,
            temperature=0.8,
            use_verifier=use_verifier,
            return_loss=return_loss,
        )
        return result

    except Exception as e:
        print(f"Error in speculative decoding: {str(e)}")
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        raise

Finally, we can try it out!

In [32]:
prompt = "What is the answer to life, the universe, and everything?"
result, loss = run_speculative_decoder(prompt, max_tokens=100, return_loss=True)
print("\nLoss:", loss)
print("\nFinal result: ", result)

Using device: cuda
Loading draft model: gpt2
Loading verifier model: gpt2-medium
Loading loss model: gpt2-medium

Generating with prompt: What is the answer to life, the universe, and everything?


<ipython-input-22-23bfba24e74d>:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



Loss: 1.2001953125

Final result:  What is the answer to life, the universe, and everything?

In this episode, we begin our journey into the universe and its mysteries. We learn about our universe, how it got here, and where we are going. We learn about the universe we live in, what the universe is, and why we are here. We learn about our existence, and why we exist. We learn about what it means to be alive, and what it means to be dead. We learn about the things we do, and how they interact with each other. We learn about the


As a test, let's also generate a completion using the draft model only without using the verifier and compare outputs and loss.

In [29]:
prompt = "What is the answer to life, the universe, and everything?"
result, draft_loss = run_speculative_decoder(prompt, max_tokens=100, return_loss=True, use_verifier=False)
print("\nLoss:", draft_loss)
print("\nFinal result: ", result)

Using device: cuda
Loading draft model: gpt2
Loading verifier model: gpt2-medium
Loading loss model: gpt2-medium

Generating with prompt: What is the answer to life, the universe, and everything?


<ipython-input-22-23bfba24e74d>:56: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():



Final result: ('What is the answer to life, the universe, and everything?\n\nIt all depends on what you\'re trying to say, but it\'s everybody\'s choice. If you have kids now, they will have kids at some point. It would be easy to say, "Okay, let\'s run a $3,500 bet on this," but if you\'re saying, "Yes, I\'ll run a $3,500 bet on this, and if I did it, there\'d be a $3,500 bonus," you\'re only saying, "Well', 2.146484375)

Loss: 2.146484375

Final result:  What is the answer to life, the universe, and everything?

It all depends on what you're trying to say, but it's everybody's choice. If you have kids now, they will have kids at some point. It would be easy to say, "Okay, let's run a $3,500 bet on this," but if you're saying, "Yes, I'll run a $3,500 bet on this, and if I did it, there'd be a $3,500 bonus," you're only saying, "Well


Note that the loss without using verifier is higher. However, this methodology is a bit flawed because we expect the loss to be lower when we're using gpt2-medium as the loss predictor above. It does tell us though that there isn't some underlying bug in the generation.

A more appropriate way to compute loss might be to use an independent model (like Eleuther's Pythia family) and compute loss or perplexity on the generated text.

##### ❓ Question #2:

How does this method offer a speed-up over traditional decoding?

The key idea is that a smaller, much faster language model can be used for generating max_new_tokens tokens. For each of these tokens, we're doing an autoregressive inference, but the model is faster than a larger LM.

Thereafter, a larger model can be used to compute the logits of the draft sequence which the smaller model produced. Since draft sequence is already generated, we only need one inference using the larger model to produce logits, and thus softmax probabilities of each token in the sequence. Then, we can threshold the these probabilities and decide whether we want to keep the draft token or sample from the distribution produced by the verifier model.

Note that we only needed to do a single inference over the larger model. Thus most of the decoding was done by the smaller model. This is what makes this algorithm efficient.

## Task 6: Guard Rails

Guard Rails is a technique for controlling the output of language models by filtering or constraining the tokens that can be generated. This provides a way to ensure outputs meet certain criteria without having to retrain or fine-tune the model.


Key components:

1. Token-Level Filtering:

- Maintains a set of blocked token IDs
- Applies filtering during the generation process
- Can handle both individual tokens and sequences


2. Vocabulary Management:

- Converts blocked words to token IDs
- Handles different forms of words (prefixes, suffixes)
- Works with the model's tokenizer


3. Probability Modification:

- Sets probabilities of blocked tokens to -inf
- Preserves relative probabilities of allowed tokens
- Integrates with temperature and top-k sampling

##### ⚠️ Hardmode Challenge:

Please implement the code required for `TokenGuardRails`.

In [ ]:
import torch
import torch.nn.functional as F

class TokenGuardRails:
    def __init__(self, tokenizer, blocked_token_ids=None):
        """
        Initialize guard rails with blocked token IDs

        Args:
            tokenizer: The tokenizer to use for converting between tokens and text
            blocked_token_ids (set): Set of token IDs that should be blocked
        """
        self.tokenizer = tokenizer
        self.blocked_token_ids = set(blocked_token_ids or [])

    def filter_logits(self, logits):
        """
        Filter logits by setting probabilities of blocked tokens to -inf

        Args:
            logits (torch.Tensor): Raw logits from model [batch_size, vocab_size]

        Returns:
            torch.Tensor: Filtered logits with same shape
        """
        mask = torch.zeros_like(logits[0]).bool()
        mask[list(self.blocked_token_ids)] = True
        logits[0, mask] = -float('inf')
        return logits

@torch.no_grad()
def generate_with_token_guard_rails(model, x, guard_rails, max_new_tokens, temperature=1.0, top_k=None):
    """
    Generate text using token-level guard rails

    Args:
        model: The language model
        x (torch.Tensor): Input token IDs [batch_size, seq_len]
        guard_rails (TokenGuardRails): Guard rails instance
        max_new_tokens (int): Maximum number of tokens to generate
        temperature (float): Sampling temperature
        top_k (int): If set, use top-k filtering
    """
    for _ in range(max_new_tokens):
      with torch.autocast("cuda"):
        if x.size(1) >= model.config.n_ctx:
          x = x[:, -model.config.n_ctx:]
        logits = model(x).logits[:, -1, :] / temperature
        logits = guard_rails.filter_logits(logits)
        probs = F.softmax(logits, dim=-1)
        if top_k is not None:
          v, _ = torch.topk(probs, k=top_k, dim=-1)
          logits[logits < v[:, -1]] = float('-inf')
        next_token = torch.multinomial(probs, num_samples=1)
        x = torch.cat((x, next_token), dim=1)
    return x

def setup_token_guard_rails(tokenizer, blocked_words):
    """Helper to set up token guard rails from word list"""
    blocked_token_ids = set()
    for word in blocked_words:
      token = tokenizer.encode(word, add_special_tokens=False)[0]
      blocked_token_ids.add(token)
    return blocked_token_ids


def run_guarded_generation(prompt, model, tokenizer, blocked_words, max_new_tokens=100):
    """Run generation with token guard rails"""
    blocked_token_ids = setup_token_guard_rails(tokenizer, blocked_words)
    guard_rails = TokenGuardRails(tokenizer, blocked_token_ids)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(model.device)
    result = generate_with_token_guard_rails(model, input_ids, guard_rails, max_new_tokens)
    return tokenizer.decode(result[0])

**Key advantages:**

- Fine-grained control over model outputs
- No need for model retraining
- Can be updated dynamically
- Maintains coherence of generated text

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# Initialize model and tokenizer
model_name = "gpt2-medium"  # Using smaller model for demonstration
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Test generation with guard rails
blocked_words = ["war", "conflict", "fight", "battle"]
prompt = "The future of international relations will be characterized by"

print("Generating with blocked words:", blocked_words)
print("\nPrompt:", prompt)
print("\nGenerating...")

result = run_guarded_generation(
    prompt=prompt,
    model=model,
    tokenizer=tokenizer,
    blocked_words=blocked_words,
    max_new_tokens=50
)

print("\nGenerated text:", result)

Generating with blocked words: ['war', 'conflict', 'fight', 'battle']

Prompt: The future of international relations will be characterized by

Generating...

Generated text: The future of international relations will be characterized by, as all market economies do, value-sequencing. A stable price, an ever-increasing quantity of money, officially fixed at the international level and from there depreciated (together with other factors of production), is necessary for the whole market


##### 👪❓ Question #4:

What other applications could Guard Rails have (specifically output Guard Rails)?

1. Guard Rails can make a model "safer" by filtering out subject matters or NSFW content by filtering out restricted keywords.
2. Aside from filtering, Guard Rails can also be used to modify (increase or decrease) the probability of tokens at specific positions, thus it can be used to make the model output adhere to a specific structure.